In [14]:
#load the data
data = open('flare').read()
#移除换行符
data = data.replace('\n','').replace('\r','')
print(data)

flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained h

In [15]:
#字符去重处理
letters = list(set(data))
print(letters)
num_letters = len(letters)
print(num_letters)

['i', 'f', 'h', 'a', 'o', 'n', 'c', 'b', 'H', 't', 'S', ' ', 'y', 'e', 'm', 'u', 'l', 'd', 'A', 's', '.', 'r', 'p']
23


In [16]:
#建立字典
#int to char
int_to_char = {a:b for a,b in enumerate(letters)}
print(int_to_char)
#char to int
char_to_int = {b:a for a,b in enumerate(letters)}
print(char_to_int)

{0: 'i', 1: 'f', 2: 'h', 3: 'a', 4: 'o', 5: 'n', 6: 'c', 7: 'b', 8: 'H', 9: 't', 10: 'S', 11: ' ', 12: 'y', 13: 'e', 14: 'm', 15: 'u', 16: 'l', 17: 'd', 18: 'A', 19: 's', 20: '.', 21: 'r', 22: 'p'}
{'i': 0, 'f': 1, 'h': 2, 'a': 3, 'o': 4, 'n': 5, 'c': 6, 'b': 7, 'H': 8, 't': 9, 'S': 10, ' ': 11, 'y': 12, 'e': 13, 'm': 14, 'u': 15, 'l': 16, 'd': 17, 'A': 18, 's': 19, '.': 20, 'r': 21, 'p': 22}


In [17]:
#time_step
time_step = 20

In [18]:
import numpy as np
from keras.utils import to_categorical
#滑动窗口提取数据
def extract_data(data, slide):    
    x = []
    y = []    
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])        
    return x,y
#字符到数字的批量转化
def char_to_int_Data(x,y, char_to_int):    
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])    
    return x_to_int, y_to_int
#实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(data, slide, num_letters, char_to_int):    
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[0], char_Data[1], char_to_int)
    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())
    Input_RESHAPED = np.array(Input).reshape(len(Input), slide)
    new = np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new, Output

In [19]:
#extract X and y from text data
X, y = data_preprocessing(data,time_step,num_letters,char_to_int)

In [20]:
print(X.shape)

(56148, 20, 23)


In [21]:
print(len(y))

56148


In [22]:
#split the data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=10)
print(X_train.shape,len(y_train))

(50533, 20, 23) 50533


In [23]:
y_train_category = to_categorical(y_train,num_letters)
print(y_train_category)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [24]:
#set up the model
from keras.models import Sequential
from keras.layers import Dense,LSTM

model = Sequential()
model.add(LSTM(units=20,input_shape=(X_train.shape[1],X_train.shape[2]),activation='relu'))
model.add(Dense(units=num_letters,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20)                3520      
                                                                 
 dense_1 (Dense)             (None, 23)                483       
                                                                 
Total params: 4,003
Trainable params: 4,003
Non-trainable params: 0
_________________________________________________________________


In [38]:
#train the model
model.fit(X_train,y_train_category,batch_size=1000,epochs=5)

Epoch 1/5
51/51 [==============================] - 2s 33ms/step - loss: 2.4560 - accuracy: 0.2963
Epoch 2/5
51/51 [==============================] - 2s 34ms/step - loss: 1.6939 - accuracy: 0.4881
Epoch 3/5
51/51 [==============================] - 2s 34ms/step - loss: 0.8400 - accuracy: 0.7720
Epoch 4/5
51/51 [==============================] - 2s 34ms/step - loss: 0.2332 - accuracy: 0.9705
Epoch 5/5
51/51 [==============================] - 2s 34ms/step - loss: 0.0637 - accuracy: 0.9992


In [39]:
#make prediction based on the training data
y_train_predict_probs = model.predict(X_train)
y_train_predict = np.argmax(y_train_predict_probs,axis=1)
print(y_train_predict)

1580/1580 [==============================] - 4s 3ms/step
[11 13  6 ... 16 13 11]


In [40]:
#transform the int to letters
y_train_predict_char = [int_to_char[i] for i in y_train_predict]
print(y_train_predict_char)

[' ', 'e', 'c', 'h', 's', 'a', 'o', ' ', ' ', 'r', 'e', 't', 'u', 'n', 'm', 'i', 'i', ' ', '.', 'd', 't', ' ', 'y', 'a', 'h', 't', '.', ' ', ' ', 't', 'u', 'r', 'c', ' ', ' ', 'e', ' ', 's', ' ', 's', 'h', 't', 'e', 'e', ' ', 's', 's', 'h', 'r', 'r', 'e', 's', 'u', ' ', 'm', 'h', 'a', 'r', 'e', 'i', 'i', ' ', 'l', 's', 'l', 'e', 'A', 'r', 'n', 'h', 'r', ' ', 'a', 'r', 'u', 'e', 'i', 'e', 'i', 'e', 't', 'm', 'a', 'a', 'r', ' ', 'i', ' ', 'u', 'H', 'H', 'n', 'h', 'p', 't', ' ', 't', 'f', 'o', ' ', 'o', 'S', ' ', ' ', ' ', 's', 'c', 't', ' ', 'y', 'a', 's', 'H', 'r', 'b', 'd', 'e', 's', ' ', 'h', 'r', 'h', 'n', 'e', 's', 'e', '.', 'e', 'a', ' ', 'n', 'a', 'r', 't', 'i', 't', ' ', 'n', ' ', 'c', 'l', 'c', 'i', 'i', ' ', 'n', 'n', 'r', 's', 's', ' ', 'c', 'e', 'l', ' ', 'r', 'u', '.', 'i', ' ', ' ', 'r', 't', 'd', 'i', 'n', 'n', 'u', 'i', 'h', 'i', 'l', 'e', 'a', 't', 'i', 't', 'l', 'A', ' ', ' ', 'c', 'l', 'i', 'r', ' ', 'e', 'i', 'e', 'i', 'h', 'a', 'd', ' ', 'a', 'i', 'm', 'a', 't', 'h',

In [41]:
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train,y_train_predict)
print(accuracy_train)

1.0


In [42]:
y_test_predict_probs = model.predict(X_test)
y_test_predict = np.argmax(y_test_predict_probs,axis=1)
accuracy_test = accuracy_score(y_test,y_test_predict)

print(accuracy_test)
print(y_test_predict)
print(y_test)


176/176 [==============================] - 0s 3ms/step
1.0
[11  2 11 ...  3  8 16]
[11, 2, 11, 4, 15, 11, 11, 16, 19, 13, 8, 21, 18, 12, 3, 5, 13, 9, 0, 2, 11, 3, 15, 2, 0, 11, 13, 22, 5, 15, 19, 0, 18, 7, 2, 3, 6, 11, 15, 20, 1, 19, 12, 15, 8, 4, 21, 5, 18, 0, 4, 11, 11, 21, 9, 1, 11, 0, 3, 19, 9, 20, 15, 17, 0, 11, 0, 3, 0, 11, 0, 20, 8, 11, 11, 3, 4, 0, 15, 13, 13, 2, 16, 11, 0, 13, 0, 9, 13, 5, 3, 18, 21, 2, 3, 2, 21, 5, 9, 19, 11, 0, 11, 19, 13, 5, 21, 11, 20, 11, 1, 16, 11, 2, 11, 9, 0, 6, 2, 9, 11, 11, 13, 2, 19, 19, 21, 8, 3, 10, 5, 9, 21, 19, 0, 5, 17, 13, 19, 13, 0, 3, 13, 20, 15, 3, 11, 6, 19, 5, 21, 0, 9, 10, 0, 20, 4, 19, 0, 5, 2, 2, 11, 11, 15, 3, 0, 5, 17, 13, 0, 9, 19, 13, 4, 3, 19, 7, 2, 11, 13, 11, 0, 3, 19, 0, 11, 13, 0, 11, 11, 12, 13, 11, 3, 10, 16, 2, 3, 11, 3, 2, 0, 0, 11, 20, 11, 5, 0, 14, 5, 3, 11, 20, 17, 10, 13, 21, 3, 21, 21, 13, 15, 19, 11, 17, 9, 15, 0, 22, 13, 0, 2, 8, 0, 0, 18, 9, 21, 11, 3, 15, 19, 13, 5, 5, 3, 13, 11, 13, 0, 13, 3, 2, 20, 0, 0, 20, 11,

In [43]:
new_letters = 'flare is a teacher in ai industry. He obtained his phd in Australia.'
X_new, y_new = data_preprocessing(new_letters,time_step,num_letters,char_to_int)
y_new_predict_probs = model.predict(X_new)
y_new_predict = np.argmax(y_new_predict_probs,axis=1)
print(y_new_predict)

2/2 [==============================] - 0s 4ms/step
[ 5 11  3  0 11  0  5 17 15 19  9 21 12 20 11  8 13 11  4  7  9  3  0  5
 13 17 11  2  0 19 11 22  2 17 11  0  5 11 18 15 19  9 21  3 16  0  3 20]


In [44]:
#transform the int to letters
y_new_predict_char = [int_to_char[i] for i in y_new_predict]
print(y_new_predict_char)

['n', ' ', 'a', 'i', ' ', 'i', 'n', 'd', 'u', 's', 't', 'r', 'y', '.', ' ', 'H', 'e', ' ', 'o', 'b', 't', 'a', 'i', 'n', 'e', 'd', ' ', 'h', 'i', 's', ' ', 'p', 'h', 'd', ' ', 'i', 'n', ' ', 'A', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a', '.']


In [45]:
for i in range(0,X_new.shape[0]-20):
    print(new_letters[i:i+20],'--predict next letter is---',y_new_predict_char[i])

flare is a teacher i --predict next letter is--- n
lare is a teacher in --predict next letter is---  
are is a teacher in  --predict next letter is--- a
re is a teacher in a --predict next letter is--- i
e is a teacher in ai --predict next letter is---  
 is a teacher in ai  --predict next letter is--- i
is a teacher in ai i --predict next letter is--- n
s a teacher in ai in --predict next letter is--- d
 a teacher in ai ind --predict next letter is--- u
a teacher in ai indu --predict next letter is--- s
 teacher in ai indus --predict next letter is--- t
teacher in ai indust --predict next letter is--- r
eacher in ai industr --predict next letter is--- y
acher in ai industry --predict next letter is--- .
cher in ai industry. --predict next letter is---  
her in ai industry.  --predict next letter is--- H
er in ai industry. H --predict next letter is--- e
r in ai industry. He --predict next letter is---  
 in ai industry. He  --predict next letter is--- o
in ai industry. He o --predict 